In [ ]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import mplotutils as mpu
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import yaml


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
with open('../path_EUR-11.txt', 'r') as file:  path_eur11 = file.read()

dir_CORDEX = path_eur11
dir_names  = f'{path_main}Scripts/Model_lists/'
dir_out    = f'{path_main}Data/EURO-CORDEX/EMT/'


## Prepare variables and parameters

In [ ]:
#Define limits for European box
lon_limits_EUR = [-10, 35]
lat_limits_EUR = [30, 70]

#Define scenarios
scenarios = ['historical', 'rcp85']

#Define models and RCPs which should be used
all_models = dict()
all_models = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models.append(eval(line[:-1]))


## Check model availability

In [ ]:
#Loop over models
for model in all_models:
    
    files = [file for file in os.listdir(dir_CORDEX) if model[0] in file and model[1] in file and model[2] in file]
    
    if not len(files)>0:
        print(model)

mod_EUR = ['_'.join(model) for model in all_models]
mod_tas = [file.split('_')[2] + '_' + file.split('_')[5] + '_' + file.split('_')[4] for file in os.listdir(dir_CORDEX) if '.nc' in file]
mod_tas = set(mod_tas)

print(sorted(list(mod_tas.difference(mod_EUR))))
print('')
print(sorted(list(set(mod_EUR).difference(mod_tas))))


## Calculate European mean temperature

In [ ]:
mod_coll = []

#Loop over scenarios
for scen in scenarios:

    print(scen)
    
    #Loop over models
    for i1, model in enumerate(all_models):

        #Skip model if no files exist
        if model[1]=='IPSL-WRF381P':
            t_freq = '_day_'
        else:
            t_freq = '_mon_'

        print("  -" + "_".join(model))

        #Get files
        files = [file for file in os.listdir(dir_CORDEX) if scen in file and model[0] in file and model[1] in file and model[2] in file and t_freq in file]
        if len(files)==0:  sys.exit('No files available for this model.')
            
        #Merge files
        files_in   = [dir_CORDEX + file for file in sorted(files)]
        file_merge = dir_out + "tas" + t_freq + "" + "_".join(model) + '_' + scen + "_tmp.nc"
        if os.path.exists(file_merge): os.remove(file_merge)
        os.system('cdo mergetime ' + " ".join(files_in) + " " + file_merge)

        #Open file
        ds = xr.open_dataset(file_merge)
        
        #Select time
        if scen=='historical':  ds = ds.sel(time=slice(None, '2005'))
        if scen=='rcp85':       ds = ds.sel(time=slice('2006', None))
        
        #Drop this variables as they cause problems with time averaging
        if 'rotated_pole' in ds:       ds = ds.drop('rotated_pole')
        if 'Lambert_Conformal' in ds:  ds = ds.drop('Lambert_Conformal')
        if 'crs' in ds:                ds = ds.drop('crs')
        
        #Resample daily data to monthly data
        if t_freq=='_day_':
            print('     Resampling day -> month')
            ds = ds.resample(time='1M').mean()
            
        #Get lat and lon names
        if 'lat' in ds.coords:   lat_name, lon_name = 'lat', 'lon'
        else:                    lat_name, lon_name = 'latitude', 'longitude'
        if 'rlat' in ds.coords:  lat_name2, lon_name2 = 'rlat', 'rlon'
        elif 'x' in ds.coords:   lat_name2, lon_name2 = 'x', 'y'
            

        #Convert longitude from [0, 360] to [-180, 180]
        if np.any(ds[lon_name]>350):  mod_coll.append(model)
        ds[lon_name] = ds[lon_name].where(ds[lon_name]<180, ((ds[lon_name] + 180) % 360) - 180)
            
        #Get mask for Europe
        mask_EUR = ((ds[lon_name]>=lon_limits_EUR[0]) & (ds[lon_name]<=lon_limits_EUR[1]) & 
                    (ds[lat_name]>=lat_limits_EUR[0]) & (ds[lat_name]<=lat_limits_EUR[1]))
        
        #Mask Europe (only for EMT!)
        ds_masked = ds.where(mask_EUR)
        
        #Weight by area
        area_weights = np.cos(ds_masked[lat_name] * np.pi/180)
        ds_EMT = ds_masked.tas.weighted(area_weights).mean((lat_name2, lon_name2))  
        
        #Calculate yearly averages
        ds_grid = ds.resample(time='1Y').mean('time')
        ds_EMT  = ds_EMT.resample(time='1Y').mean('time')
        ds_EMT  = ds_EMT.to_dataset(name='tas')

        #Save data in file
        time_str  = str(ds.time[0].dt.year.values) + '-' + str(ds.time[-1].dt.year.values)
        fname_out_EMT  = dir_out + 'EMT_year_' + scen + '_' + "_".join(model) + '_' + time_str + ".nc"
        fname_out_grid = dir_out + 'tas_year_' + scen + '_' + "_".join(model) + '_' + time_str + ".nc"
        if os.path.exists(fname_out_EMT):   os.remove(fname_out_EMT)
        if os.path.exists(fname_out_grid):  os.remove(fname_out_grid)
        ds_EMT.to_netcdf(fname_out_EMT)
        ds_grid.to_netcdf(fname_out_grid)
        
        #Remove temporary merge file
        os.remove(file_merge)


## Plot data and box

In [ ]:
fig1, ax1 = plt.subplots(1, 1, figsize=(14,6), subplot_kw=dict(projection=ccrs.PlateCarree()))
ax1.coastlines(linewidth=0.75)

for i1, model in enumerate(all_models):
    
    files = [file for file in os.listdir(dir_CORDEX) if model[0] in file and model[1] in file and model[2] in file and 'tas_' in file]
    if len(files)==0:
        print(model)
        continue
        
#     print(model)

    file = files[0]
    ds = xr.open_dataset(dir_CORDEX + file)
    ds = ds['tas'].isel(time=0)
    
    ds = ds * 0 + i1
    
    if 'lat' in ds.coords:  lat_name_p, lon_name_p = 'lat', 'lon'
    else:                   lat_name_p, lon_name_p = 'latitude', 'longitude'

    ax1.pcolormesh(ds[lon_name_p], ds[lat_name_p], ds, vmin=0, vmax=len(all_models), shading='auto', transform=ccrs.PlateCarree(), alpha=0.1)

# Draw the rectangular extent of the second plot on the first:
x = [-10, 35, 35, -10, -10]
y = [30, 30, 70, 70, 30]
ax1.fill(x, y, transform=ccrs.PlateCarree(), edgecolor='r', facecolor='none', linewidth=3);
